In [1]:
################ convert pdf to images ########################

from pdf2image import convert_from_path
import os

def pdf_to_images(pdf, filename):
    pdf = pdf.replace('\\', '\\\\')
    # print(pdf)
    try:
        images = convert_from_path(pdf)
        for i in range(len(images)):
            images[i].save('C:\\air_ticket\\comakeit_electricity\\images\\{}_page{}'.format(filename, i) +'.jpg', 'JPEG')
    except:
        pass

path ="C:\\air_ticket\\comakeit_electricity\\DKH"

#store all the file names in this list
filelist = []

for root, dirs, files in os.walk(path):
	for file in files:
        #append the file name to the list
		filelist.append(os.path.join(root,file))


# print all the file names
for file in filelist:
    filename = os.path.split(file)[-1].split('.pdf')[0]
    if ' ' in filename:
        filename = filename.replace(' ', '_')
    # print(filename)
    try:
        pdf_to_images(file, filename)
    except:
        pass

In [4]:
########################## parse output of preprocessed images #############################

from PIL import Image, ImageEnhance
import cv2
import numpy as np
from skimage import io
from skimage.transform import rotate
from skimage.color import rgb2gray
from deskew import determine_skew
from matplotlib import pyplot as plt
from pdf2image import convert_from_path
import shutil
import glob
import os

def set_dpi(image, preproc_image):
    im = Image.open(image)
    im.save(preproc_image, dpi=(600,600))
    return preproc_image

def increase_contrast(image, preproc_image):
    im = Image.open(image)
    enhancer = ImageEnhance.Contrast(im)
    factor = 2 #increase contrast
    im_output = enhancer.enhance(factor)
    im_output.save(preproc_image)
    return preproc_image

def increase_sharpness(image, preproc_image):
    im = Image.open(image)
    enhancer = ImageEnhance.Sharpness(im)
    factor = 2
    im_output = enhancer.enhance(factor)
    im_output.save(preproc_image)
    return preproc_image

def convert_to_grayscale(image, preproc_image):
    im = cv2.imread(image)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    cv2.imwrite(preproc_image, im)
    return preproc_image

def convert_to_rgb(image, preproc_image):
    im = cv2.imread(image)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    cv2.imwrite(preproc_image, im)
    return preproc_image

def otsu_threshold(image, preproc_image):
    im = cv2.imread(image)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    # im = cv2.threshold(im, 120, 255, cv2.THRESH_BINARY, cv2.THRESH_OTSU)[1]
    im = cv2.adaptiveThreshold(im,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
    cv2.imwrite(preproc_image, im)
    return preproc_image

def remove_noise(image, preproc_image):
    im = cv2.imread(image)
    im = cv2.fastNlMeansDenoising(im, None, 20, 7, 21)
    cv2.imwrite(preproc_image, im)
    return preproc_image

def deskew(image):
    im = io.imread(image)
    angle = determine_skew(im)
    rotated = rotate(im, angle, resize=True) * 255
    io.imsave(image, rotated.astype(np.uint8))
    return image

combinations = ['set_dpi-increase_contrast-convert_to_grayscale', 'set_dpi-convert_to_grayscale', 'set_dpi-increase_contrast-convert_to_grayscale-remove_noise']
# combinations = ['set_dpi-increase_contrast-increase_sharpness']

for image in glob.glob('C:\\air_ticket\\comakeit_electricity\\images\\*.jpg'):
    # filename = os.path.split(image)[-1].split('.jpg')[0]
    # print(filename)
    # preproc_image = 'preprocessed_ensemble\\'+filename+'.jpg'
    # shutil.copy(image, preproc_image)
    for combination in combinations:
        filename = os.path.split(image)[-1].split('.jpg')[0]
        preproc_image = 'C:\\air_ticket\\comakeit_electricity\\preprocessed_images\\'+filename+'.jpg'
        shutil.copy(image, preproc_image)
        methods = combination.split('-')
        for method in methods:
            if 'deskew' in method:
                preproc_image = eval(method+'(preproc_image)')
            else:
                preproc_image = eval(method+'(preproc_image, preproc_image)')
        # print(combination)
        image_name = 'C:\\air_ticket\\comakeit_electricity\\preprocessed_images\\'+filename+'_'+combination+'.jpg'
        img = cv2.imread(preproc_image)
        cv2.imwrite(image_name, img)

        psm_values = [3, 4]

        for psm in psm_values:

            parsed_output = 'C:\\air_ticket\\comakeit_electricity\\preprocessed_images\\'+filename+'_'+combination+'_'+str(psm)
            command = 'tesseract {} {} -l eng --psm {}'.format(preproc_image, parsed_output, psm)
            os.system(command)

In [ ]:
increase dpi 300, increase contrast, convert to grayscale, denoise

In [5]:
################### ensemble the output files #####################

import glob
import os

def ensemble_output():

    for file in glob.glob('C:\\air_ticket\\comakeit_electricity\\preprocessed_images\\*.txt'):
        filename = os.path.split(file)[-1].split('.txt')[0]
        ind = filename.index('page')
        filename = filename[:ind+5]
        print(filename)
        with open(file, encoding="utf-8") as fp:
                # read the file
                text = fp.read()
                # open new file and append text of files for the same name 
                with open('C:\\air_ticket\\comakeit_electricity\\ensemble_files\\{}.txt'.format(filename), 'a') as file:
                    # to separate the text of multiple configuration using ---
                    file.write(text+"\n---"+"\n")

ensemble_output()


01_Electricity_Bill_January_page0
01_Electricity_Bill_January_page0
01_Electricity_Bill_January_page0
01_Electricity_Bill_January_page0
01_Electricity_Bill_January_page0
01_Electricity_Bill_January_page0
02_Electricity_Bill_February_page0
02_Electricity_Bill_February_page0
02_Electricity_Bill_February_page0
02_Electricity_Bill_February_page0
02_Electricity_Bill_February_page0
02_Electricity_Bill_February_page0
03_Electricity_Bill_March_page0
03_Electricity_Bill_March_page0
03_Electricity_Bill_March_page0
03_Electricity_Bill_March_page0
03_Electricity_Bill_March_page0
03_Electricity_Bill_March_page0
04_Electricity_Bill_April_page0
04_Electricity_Bill_April_page0
04_Electricity_Bill_April_page0
04_Electricity_Bill_April_page0
04_Electricity_Bill_April_page0
04_Electricity_Bill_April_page0
05_Electricity_Bill_May_page0
05_Electricity_Bill_May_page0
05_Electricity_Bill_May_page0
05_Electricity_Bill_May_page0
05_Electricity_Bill_May_page0
05_Electricity_Bill_May_page0
06_Electricity_Bill_fo